In [7]:
import json
import pickle
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import deepchem as dc
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import MolFromSmiles
from deepchem.feat import RDKitDescriptors   
import networkx as nx

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model

In [2]:
# Define the seq_to_cat function
seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
seq_dict = {v:i for i,v in enumerate(seq_voc)}
seq_dict_len = len(seq_dict)
max_seq_len = 1000
def seq_to_cat(prot):
    x = np.zeros(max_seq_len)
    for i, ch in enumerate(prot[:max_seq_len]): 
        x[i] = seq_dict[ch]
    return x  

In [15]:
# Load in the drugs to be tested 
fpath = 'data/kiba/'
drugs_ = json.load(open(fpath + "ligands_can.txt"), object_pairs_hook=OrderedDict)
drugs = np.array([ Chem.MolToSmiles(Chem.MolFromSmiles(d),isomericSmiles=True) for d in drugs_.values() ])
drugs_2 = np.array([Chem.MolFromSmiles(smile) for smile in drugs ])
drugs_ecfp2 = np.array([ Chem.GetMorganFingerprintAsBitVect(m,2) for m in drugs_2 ])

drug_scaler = StandardScaler()
drug_scaler.fit(drugs_ecfp2)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
# Load in the protein dataset in order to find the proper scaling trsnform for the covid_protein
proteins_ = json.load(open(fpath + "proteins.txt"), object_pairs_hook=OrderedDict)
proteins = np.array(list(proteins_.values()))
proteins = np.array([seq_to_cat(p) for p in proteins])

protein_scaler = StandardScaler()
protein_scaler.fit(proteins)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [17]:
# Load in the covid protein
covid_protein_path = 'data/6Y84_A.fasta.txt'
with open(covid_protein_path) as f:
    file_txt = f.readlines()
    
covid_protein_str = ''
covid_protein_str = covid_protein_str.join([line.strip() for line in file_txt[1:]])

In [24]:
# Get data ready for the model
drugs_ecfp2 = drug_scaler.transform(drugs_ecfp2)
drugs_ecfp2 = drugs_ecfp2[:,:,np.newaxis]

covid_protein = np.array([seq_to_cat(covid_protein_str)])
covid_proteins = protein_scaler.transform(np.repeat(covid_protein, 2111, axis=0))
covid_proteins = covid_proteins[:,:,np.newaxis]

In [27]:
# Load the trained model
model = keras.models.load_model('final_model.h5')

In [47]:
# Create binding predictions
predictions = model.predict([drugs_ecfp2, covid_proteins], batch_size=1)

In [58]:
# Get the drugs with the 5 highest binding affinities
drug_dict = json.load(open(fpath + "ligands_can.txt"), object_pairs_hook=OrderedDict)
drug_names = list(drug_dict.keys())
max_idxs = np.argsort(predictions[:,0])[-5:]

print('Top 5 predicted Drugs\n')
for idx in max_idxs[::-1]:
    idx = int(idx)
    affinity = predictions[idx]
    drug_name = drug_names[idx]
    print(f'Drug {drug_name} has a predicted binding affinity of: {affinity}')

Top 5 predicted Drugs

Drug CHEMBL291126 has a predicted binding affinity of: [155.24535]
Drug CHEMBL1971227 has a predicted binding affinity of: [79.53979]
Drug CHEMBL2086760 has a predicted binding affinity of: [75.149315]
Drug CHEMBL483525 has a predicted binding affinity of: [74.68852]
Drug CHEMBL1822792 has a predicted binding affinity of: [72.61388]


In [61]:
# Look at results of cross validation
cross_val = np.loadtxt('5_fold_results')
print('MSE results for 5-fold cross validaton:\n',cross_val)

MSE results for 5-fold cross validaton:
 [0.43387262 0.56399456 0.45285764 0.4416108  0.6529016 ]
